In [1]:
%load_ext google.cloud.bigquery
%matplotlib inline

# some pandas options
import pandas as pd
pd.set_option("display.expand_frame_repr", False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

sats = 100000000

In [2]:
%%bigquery raw
with b as (
    select block_timestamp_month
    , date(block_timestamp) as day
    , block_number
    , count(*) as group_total
    from `bigquery-public-data.crypto_bitcoin.transactions`
  WHERE TRUE
  AND is_coinbase IS FALSE
  AND block_timestamp_month = '2021-06-01'
    group by 1,2,3

)
select a.*, b.group_total from (
select block_timestamp_month, day, block_number, tx_label
     , avg(avg_in)/100000000 avg_in_btc
     , avg(avg_out)/100000000 avg_out_btc
     , avg(median_in_btc) m_in, avg(median_out_btc) m_out
     , count(*) as label_total
  from (
select *
       , (percentile_cont(avg_in, 0.5) over(partition by block_timestamp_month, day, block_number, tx_label))/100000000 median_in_btc
     , (percentile_cont(avg_out, 0.5) over(partition by block_timestamp_month, day, block_number, tx_label))/100000000 median_out_btc
from (
select *, case when (i_l = total and o_l = total) 
                 or (i_ls = total and o_ls = total)
                 or (i_s = total and o_s = total) then 'same_address_type'
               when (o_l = total) or (o_s = total) or (o_ls = total) then 'diff_input_type_one_output_type'
               when (i_l = total) or (i_s = total) or (i_ls = total) then 'single_input_type_diff_output_type'
          else 'diff_input_type_diff_output_type' end as tx_label
 from (
select `hash`, day, block_number, block_timestamp_month, count(*) as total
     , sum(i_legacy) i_l
     , sum(i_legacy_script) i_ls
     , sum(i_segwit) i_s
     , sum(o_legacy) o_l
     , sum(o_legacy_script) o_ls
     , sum(o_segwit) o_s
     , avg(input_value) as avg_in, avg(output_value) as avg_out
     
  FROM (
select *
     , case when left(ia, 1) = '1' then 1 else 0 end as i_legacy
     , case when left(ia, 1) = '3' then 1 else 0 end as i_legacy_script
     , case when left(ia, 3) = 'bc1' then 1 else 0 end as i_segwit
     , case when left(oa, 1) = '1' then 1 else 0 end as o_legacy
     , case when left(oa, 1) = '3' then 1 else 0 end as o_legacy_script
     , case when left(oa, 3) = 'bc1' then 1 else 0 end as o_segwit
  FROM (
SELECT DISTINCT block_timestamp_month, date(block_timestamp) as day
     , block_number
     , `hash`
     , ia
     , oa
     , input_value
     , output_value
  FROM
  `bigquery-public-data.crypto_bitcoin.transactions`
    , UNNEST(inputs) AS i
    , UNNEST(i.addresses) as ia
    , UNNEST(outputs) AS o
    , UNNEST(o.addresses) AS oa
WHERE TRUE
  AND is_coinbase IS FALSE
  AND block_timestamp_month >= '2021-01-01'
  AND output_count < 3
  )) group by 1,2,3,4 ))) group by 1,2,3,4) a join b on 
  a.block_timestamp_month = b.block_timestamp_month and
  a.day = b.day and
  a.block_number = b.block_number

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14069/14069 [00:03<00:00, 4277.03rows/s]


In [5]:
%%bigquery test
select *, case when (i_l = total and o_l = total) 
                 or (i_ls = total and o_ls = total)
                 or (i_s = total and o_s = total) then 'same_address_type'
               when (o_l = total) or (o_s = total) or (o_ls = total) then 'diff_input_type_one_output_type'
               when (i_l = total) or (i_s = total) or (i_ls = total) then 'single_input_type_diff_output_type'
          else 'diff_input_type_diff_output_type' end as tx_label
 from (
select `hash`, day, block_number, block_timestamp_month, count(*) as total
     , sum(i_legacy) i_l
     , sum(i_legacy_script) i_ls
     , sum(i_segwit) i_s
     , sum(o_legacy) o_l
     , sum(o_legacy_script) o_ls
     , sum(o_segwit) o_s
     , avg(input_value) as avg_in, avg(output_value) as avg_out
     
  FROM (
select *
     , case when left(ia, 1) = '1' then 1 else 0 end as i_legacy
     , case when left(ia, 1) = '3' then 1 else 0 end as i_legacy_script
     , case when left(ia, 3) = 'bc1' then 1 else 0 end as i_segwit
     , case when left(oa, 1) = '1' then 1 else 0 end as o_legacy
     , case when left(oa, 1) = '3' then 1 else 0 end as o_legacy_script
     , case when left(oa, 3) = 'bc1' then 1 else 0 end as o_segwit
  FROM (
SELECT DISTINCT block_timestamp_month, date(block_timestamp) as day
     , block_number
     , `hash`
     , ia
     , oa
     , input_value
     , output_value
  FROM
  `bigquery-public-data.crypto_bitcoin.transactions`
    , UNNEST(inputs) AS i
    , UNNEST(i.addresses) as ia
    , UNNEST(outputs) AS o
    , UNNEST(o.addresses) AS oa
WHERE TRUE
  AND is_coinbase IS FALSE
  AND block_timestamp_month >= '2021-01-01'
  AND output_count < 3
  )) group by 1,2,3,4 ) limit 1000


Query is running:   0%|                                                                                                                                              | 0/1 [00:00<?, ?query/s]
Query executing stage S00: Input and status COMPLETE : 0.00s:   0%|                                                                                                 | 0/16 [00:00<?, ?query/s]
Query complete after 0.00s: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 8991.01query/s]

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 418.99rows/s]


In [6]:
test.head( n=20)

,hash,day,block_number,block_timestamp_month,total,i_l,i_ls,i_s,o_l,o_ls,o_s,avg_in,avg_out,tx_label
0,19ab2d5cf252ec5444d441b7b6a8be1a90870b8ef973e12fd3257ab916dbb194,2021-06-27,688949,2021-06-01,8,0,8,0,0,8,0,132640356.000000000,132602269.000000000,same_address_type
1,b167913d552ebe4b85a40fad94655797edb4448d9560f66ab14ca8fc7236e173,2021-06-20,688258,2021-06-01,782,0,782,0,0,0,782,2317874968.000000000,2317002304.000000000,diff_input_type_one_output_type
2,80430f749bbb2a6b4a490d52fec49319c5325b80a6a41b046715d994d5fc6fe7,2021-10-04,703520,2021-10-01,1914,0,1912,2,0,1914,0,20100167.000000000,20005901.000000000,diff_input_type_one_output_type
3,576715aedee92f1c2e3b22be1141f662a6babc1d4dbfb9b244afca8a26c2e3d3,2021-03-19,675311,2021-03-01,2,2,0,0,2,0,0,1334312.000000000,1325312.000000000,same_address_type
4,ed3ed824ac220ba880ae4f74d0d5a44e34b2c82ced08e987fb0262f5e7443150,2021-04-08,678335,2021-04-01,1,1,0,0,1,0,0,1882371.000000000,1838180.000000000,same_address_type
5,52ba8e35eb9aaeba2915738357fd517882a3d7d62abf890392531125afab0789,2021-10-24,706399,2021-10-01,4,0,0,4,2,0,2,1787694.000000000,1787154.000000000,single_input_type_diff_output_type
6,8faa43681effa52d9f133f32236744deadb9f44e6d222e437dae226cb87f5b05,2021-08-06,694391,2021-08-01,6,0,0,6,0,3,3,240884.000000000,240146.000000000,single_input_type_diff_output_type
7,d9720c02880a184504362b1ff5cca596573f07a36266ecca2b6738b33f9832a2,2021-10-23,706294,2021-10-01,2,0,0,2,1,0,1,9235372.000000000,9235036.000000000,single_input_type_diff_output_type
8,c629fc506e8d3bac707f5b34bfe2eb2a9de1ac9e504741d846386d38fa944e4d,2021-10-11,704488,2021-10-01,2,0,2,0,0,2,0,85166668.000000000,85165399.000000000,same_address_type
9,46b5643cfff2303a58c5210238239c5bcf023bda179ea8ab291e9a975db38fb8,2021-09-30,702799,2021-09-01,350,350,0,0,0,350,0,50053768.000000000,50000000.000000000,diff_input_type_one_output_type


In [44]:
%%bigquery cols
select *

  FROM
  `bigquery-public-data.crypto_bitcoin.transactions`

WHERE TRUE
  AND is_coinbase IS FALSE
  AND block_timestamp_month = '2021-06-01'
  AND output_count < 3
      
 limit 100

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 32.83rows/s]


# address type heuristics

In the process of creating a bitcoin transaction, the wallet software will often generate a "change address." This is the amount to be returned to the sender of the transaction, i.e to an address owned by the same wallet that is constructing the transaction. If we can determine which of the outputs of a transaction is the change address, we can then deduce which output was the actual payment.

Bitcoin allows for three address types as of today: P2KH, P2SH, and bech32. A user normally has to specify which address format they want to use when setting up a wallet, so it is reasonable to assume all address generated by a certain wallet will be of the same type. We can use this to determine a change address in the following way:

1. Find transactions with two outputs where the output types are not the same
2. Compare the input address types and find the output which matches the input address type

For example, if wallet A is bech32 and I am making a payment to wallet B which is P2KH, one of the outputs will be to their P2KH address and my wallet will generate a new bech32 address for the change, which will come from the second output of the transaction.

In [29]:
%%bigquery raw_tx
SELECT block_timestamp_month
     , date(block_timestamp) as day
     , block_number
     , `hash`
     , input_count
     , output_count
     , inputs
     , outputs
     , input_value
     , output_value
     , input_value - output_value as mining_fee
  FROM
  `bigquery-public-data.crypto_bitcoin.transactions`
WHERE TRUE
  AND is_coinbase IS FALSE
  -- AND block_number between 700000 and 700010
  AND date(block_timestamp) = '2021-06-01'
  AND output_count < 3

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 209309/209309 [03:00<00:00, 1158.85rows/s]


In [30]:
txs = raw_tx.groupby(['hash'])

In [31]:
def label_inputs_and_outputs(df):
    tx_type = {
        'witness_v0_keyhash': 0,
        'witness_v0_scripthash': 0,
        'pubkeyhash': 0,
        'scripthash': 0,
        'nonstandard': 0,
        'multisig': 0,
    }
    # verify inputs are all the same
    for tx_input in df['inputs'].values[0]:
        tx_type[tx_input['type']] += 1
        
    for k,v in tx_type.items():
        if k == 'nonstandard' and v > 0:
            df['input_type'] = k
            break
        elif v == df['input_count'].values[0]:
            df['input_type'] = k
            break
        else:
            df['input_type'] = 'mixed_inputs'
    
    tx_type = tx_type.fromkeys(tx_type, 0)
    # check out types
    for tx_output in df['outputs'].values[0]:
        tx_type[tx_output['type']] += 1
    for k,v in tx_type.items():
        if k == 'nonstandard' and v > 0:
            df['output_type'] = k
            break
        elif v == df['output_count'].values[0]:
            df['output_type'] = k
            break
        else:
            df['output_type'] = 'mixed_outputs'
    return df

In [33]:
def identify_change_address(df):
    def f(v):
        return df[v].values[0]
    
    if f('input_count') == 1 and f('output_count') == 1:
        df['change_value'] = 0
    elif f('output_type') == 'nonstandard':
        for output in f('outputs'):
            if output['type'] != 'nonstandard':
                df['change_value'] = 0
    elif f('output_type') == 'mixed_outputs' and f('output_count') > 1:
        for output in f('outputs'):
            if output['type'] == f('input_type'):
                df['change_value'] = output['value']
    elif f('output_count') > 1 and f('input_count') > 1:
        min_input = min(df['inputs'].values[0], key=lambda x: x['value'])['value']
        change_value = list(filter(lambda x: x['value'] <= min_input, f('outputs')))
        if len(change_value) == 1:
            # what about when it's more??
            df['change_value'] = change_value[0]['value']
        elif len(change_value) > 1:
            min_input = min(df['inputs'].values[0], key=lambda x: x['value'])['value'] - f('mining_fee')
            change_value = list(filter(lambda x: x['value'] <= min_input, f('outputs')))
    elif f('input_count') > 1 and f('output_count') == 1:
        df['change_value'] = 0
    elif f('input_count') == 1:
        for output in f('outputs'):
            if output['addresses'][0] == f('inputs')[0]['addresses'][0]:
                df['change_value'] = output['value']
    return df
                
labeled = txs.apply(label_inputs_and_outputs)
change_value = labeled.groupby(['hash']).apply(identify_change_address)

change_value['payment_value'] = change_value['output_value'] - change_value['change_value']
final = change_value[['hash','block_timestamp_month','day','block_number','input_count','output_count','input_value','output_value','input_type','output_type','change_value','payment_value']]

In [34]:
final.to_csv('tx_with_change_address_2021-06-01.csv', index=False)

In [48]:
final[final['output_type'] != 'nonstandard']['change_value'].fillna(0).astype(int).describe()

count    2.065590e+05
mean     2.948768e+08
std      1.408675e+10
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.034959e+06
max      1.028214e+12
Name: change_value, dtype: float64

In [122]:
%%bigquery info_schema

SELECT
* EXCEPT(is_generated, generation_expression, is_stored, is_updatable)
FROM
`bigquery-public-data`.crypto_bitcoin.INFORMATION_SCHEMA.COLUMNS
WHERE
table_name="transactions"

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:02<00:00,  7.58rows/s]


In [123]:
info_schema

,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_hidden,is_system_defined,is_partitioning_column,clustering_ordinal_position
0,bigquery-public-data,crypto_bitcoin,transactions,hash,1,NO,STRING,NO,NO,NO,NaN
1,bigquery-public-data,crypto_bitcoin,transactions,size,2,YES,INT64,NO,NO,NO,NaN
2,bigquery-public-data,crypto_bitcoin,transactions,virtual_size,3,YES,INT64,NO,NO,NO,NaN
3,bigquery-public-data,crypto_bitcoin,transactions,version,4,YES,INT64,NO,NO,NO,NaN
4,bigquery-public-data,crypto_bitcoin,transactions,lock_time,5,YES,INT64,NO,NO,NO,NaN
5,bigquery-public-data,crypto_bitcoin,transactions,block_hash,6,NO,STRING,NO,NO,NO,NaN
6,bigquery-public-data,crypto_bitcoin,transactions,block_number,7,NO,INT64,NO,NO,NO,NaN
7,bigquery-public-data,crypto_bitcoin,transactions,block_timestamp,8,NO,TIMESTAMP,NO,NO,NO,NaN
8,bigquery-public-data,crypto_bitcoin,transactions,block_timestamp_month,9,NO,DATE,NO,NO,YES,NaN
9,bigquery-public-data,crypto_bitcoin,transactions,input_count,10,YES,INT64,NO,NO,NO,NaN


In [ ]:
september 2nd - half are 2 out, 1 in